In [ ]:
import pygame
import numpy as np
import math,os
from swerve_drive import SwerveDrive
from PIDController import PIDController
from TrajectoryParser import AutoParser
from swerve_environment import SwerveEnvironment
from config import *
from Genetic_Algo import GeneticAlgorithm

In [ ]:
# Create environment
start_point = (4,4,0)
env_HW = (8.21, 16.54)  # FRC field dimensions in meters (height, width)
env = SwerveEnvironment(width=1000, height=800, dt=0.02,HW = env_HW, start_point = start_point)


In [ ]:

auto_parser = AutoParser(
    r"c:\Users\chris.reckner\Documents\pythontests\Machine Learning\pathplanner\autos\Wonky 3.auto",
    paths_directory=r"c:\Users\chris.reckner\Documents\pythontests\Machine Learning\pathplanner\paths"
)

print(f"\nPath names in auto: {auto_parser.get_path_names()}")

# Get summary
summary = auto_parser.get_trajectory_summary()
print(f"\nAuto Summary:")
print(f"  Number of paths: {summary['num_paths']}")
print(f"  Total duration: {summary['duration']:.2f} seconds")
print(f"  Total distance: {summary['total_distance']:.2f} meters")
print(f"  Start position: ({summary['start_position'][0]:.2f}, {summary['start_position'][1]:.2f})")
print(f"  End position: ({summary['end_position'][0]:.2f}, {summary['end_position'][1]:.2f})")

# Generate combined trajectory
combined_trajectory = auto_parser.generate_combined_trajectory(dt=0.25)



In [ ]:
def evaluate_individual(individual, render=True,
                        ga_instance=None, current_gen=None, current_individual=None, episodes=1):
    # Create PID controllers (assuming they are defined elsewhere)
    accumulated_error = 0
    for ep in range(episodes):
        trans_pidx = PIDController(kp=individual[0], ki=individual[1], kd=individual[2])
        trans_pidy = PIDController(kp=individual[0], ki=individual[1], kd=individual[2])
        angular_pid = PIDController(kp=individual[3], ki=individual[4], kd=individual[5])

        success, trajectory_data = env.run_trajectory(combined_trajectory, trans_pidx, trans_pidy, angular_pid,render=render)
        for data_point in trajectory_data:
            accumulated_error += data_point['error'][0] + data_point['error'][1]
            
        return accumulated_error / episodes

In [ ]:
ga = GeneticAlgorithm(
    population_size=POPULATION_SIZE,
    generations=GENERATIONS,
    visualize_best=VISUALIZE_BEST,
    evaluate_individual=evaluate_individual,
    parameter_ranges=PID_RANGES)